### Intersection of mrcanavar, PacBio CCS, and ONT high coverage regions to identify potential CNV

Generating excessive coverage bed files using calculations from mosdepth for both PacBio CCS 15kb_20kb merged and ONT bam files.

ultra-long-ont_hs37d5_phased_reheader.bam is from ftp://ftp-trace.ncbi.nlm.nih.gov/giab/ftp/data/AshkenazimTrio/HG002_NA24385_son/Ultralong_OxfordNanopore/guppy-V2.3.4_2019-06-26/ultra-long-ont_hs37d5_phased.bam

HG002.SequelII.merged_15kb_20kb.pbmm2.hs37d5.haplotag.RTG.10x.trio.bam is from ftp://ftp-trace.ncbi.nlm.nih.gov/giab/ftp/data/AshkenazimTrio/HG002_NA24385_son/PacBio_CCS_15kb_20kb_chemistry2/GRCh37/HG002.SequelII.merged_15kb_20kb.pbmm2.hs37d5.haplotag.RTG.10x.trio.bam



In [ ]:
samtools view -1 -F 0x100 ultra-long-ont_hs37d5_phased_reheader.bam -h > filtered_ultra-long-ont_hs37d5_phased_reheader.bam

mosdepth -b 1000 -x --no-per-base HG002.SequelII.merged_15kb_20kb_1000_window_size HG002.SequelII.merged_15kb_20kb.pbmm2.hs37d5.haplotag.RTG.10x.trio.bam

mosdepth -b 1000 -x --no-per-base HG002_GRCh37.ultra_long_ONT_1000_window_size filtered_ultra-long-ont_hs37d5_phased_complete.bam


chr_1_22 <- seq(1,22)
mosdepth_CCS_15kb_20kb_merged_1000_window_size = read.delim("HG002.SequelII.merged_15kb_20kb_1000_window_size.regions.bed", col.names = c("CHR","START","END","DEPTH"))
mosdepth_CCS_15kb_20kb_merged_1000_window_size_chr_1_22 <- mosdepth_CCS_15kb_20kb_merged_1000_window_size[which(mosdepth_CCS_15kb_20kb_merged_1000_window_size[,"CHR"] %in% chr_1_22),]
quantile(mosdepth_CCS_15kb_20kb_merged_1000_window_size_chr_1_22[,"DEPTH"]): 

      0%      25%      50%      75%     100% 

    0.00    48.12    55.21    61.68 26903.95 

IQR(mosdepth_CCS_15kb_20kb_merged_1000_window_size_chr_1_22[,"DEPTH"]): 13.56

(quantile(mosdepth_CCS_15kb_20kb_merged_1000_window_size_chr_1_22[,"DEPTH"])[3]/2)*2.5 : 69.0125


mosdepth_ONT_1000_window_size = read.delim("HG002_GRCh37.ultra_long_ONT_1000_window_size.regions.bed", col.names = c("CHR","START","END","DEPTH"))
mosdepth_ONT_1000_window_size_chr_1_22 <- mosdepth_ONT_1000_window_size[which(mosdepth_ONT_1000_window_size[,"CHR"] %in% chr_1_22),]
quantile(mosdepth_ONT_1000_window_size_chr_1_22[,"DEPTH"])

    0%      25%      50%      75%     100% 

    0.00    52.36    59.13    65.46 20550.08 

IQR(mosdepth_ONT_1000_window_size_chr_1_22[,"DEPTH"]): 13.1

(quantile(mosdepth_ONT_1000_window_size_chr_1_22[,"DEPTH"])[3]/2)*2.5: 73.9125 

In [ ]:

python convert_mosdepth_to_excessive_coverage.py --input HG002_GRCh37_CCS_merged_15kb_20kb_1000_window_size.regions.bed --output HG002_GRCh37_CCS_merged_15kb_20kb_1000_window_size.regions_excessive_coverage_cnv_threshold.bed --threshold 69.0125

bedtools intersect -a AJtrio-HG002.hs37d5.300x.bam.bilkentuniv.052119.dups.bed -b HG002_GRCh37_CCS_merged_15kb_20kb_1000_window_size.regions_excessive_coverage_cnv_threshold.bed > mrcanavar_intersect_ccs_cnv_threshold.bed

python convert_mosdepth_to_excessive_coverage.py --input HG002_GRCh37.ultra_long_ONT_1000_window_size.regions.bed --output HG002_GRCh37.ultra_long_ONT_1000_window_size.regions_excessive_coverage_cnv_threshold.bed --threshold 73.9125

bedtools intersect -a mrcanavar_intersect_ccs_1000_window_size_cnv_threshold.bed -b HG002_GRCh37.ultra_long_ONT_1000_window_size.regions_excessive_coverage_cnv_threshold.bed > mrcanavar_intersect_ccs_1000_window_size_cnv_threshold_intersect_ont_1000_window_size_cnv_threshold.bed


convert_mosdepth_to_excessive_coverage.py 

import argparse

parser = argparse.ArgumentParser(description="Subset bed file to callable regions only")
parser.add_argument('--input_file', metavar="I", type=str, nargs="+", help="input bed file")
parser.add_argument('--output_file', metavar="O", type=str, nargs="+", help="output file")
parser.add_argument('--threshold', metavar="T", type=str, nargs="+", help="input threshold")
args = parser.parse_args()

f = open(args.input_file[0], "r") 
f_lines = f.readlines()

f_out = open(args.output_file[0], "w+")
threshold = float(args.threshold[0])

for line in f_lines:   
    if "DEPTH" in line: 
        continue
    line_split = line.split("\t")
    depth_field = float(line_split[3])
    if depth_field > threshold:
        f_out.write(line)
        f_out.flush()  

f.close()
f_out.close()

### Union of CCS and ONT CNV outlier coverage difficult bed to calculate quantiles and IQR for CCS and ONT

mosdepth_CCS_15kb_20kb_merged_1000_window_size = read.delim("HG002_GRCh37_CCS_merged_15kb_20kb_1000_window_size.regions.bed", col.names = c("CHR","START","END","DEPTH"))

mosdepth_ONT_1000_window_size = read.delim("HG002_GRCh37.ultra_long_ONT_1000_window_size.regions.bed", col.names = c("CHR","START","END","DEPTH"))

chr_1_22 <- seq(1,22)
mosdepth_CCS_15kb_20kb_merged_1000_window_size_chr_1_22 <- mosdepth_CCS_15kb_20kb_merged_1000_window_size[which(mosdepth_CCS_15kb_20kb_merged_1000_window_size[,"CHR"] %in% chr_1_22),]

mosdepth_ONT_1000_window_size_chr_1_22 <- mosdepth_ONT_1000_window_size[which(mosdepth_ONT_1000_window_size[,"CHR"] %in% chr_1_22),]

df_mosdepth_CCS_15kb_20kb_merged_1000_window_size <- data.frame(mosdepth_CCS_15kb_20kb_merged_1000_window_size_chr_1_22)
df_mosdepth_ONT_1000_window_size <- data.frame(mosdepth_ONT_1000_window_size_chr_1_22)

df_mosdepth_CCS_15kb_20kb_merged_1000_window_size_ONT_1000_combined <- df_mosdepth_CCS_15kb_20kb_merged_1000_window_size

df_mosdepth_CCS_15kb_20kb_merged_1000_window_size_ONT_1000_combined[,5] <-  mosdepth_ONT_1000_window_size_chr_1_22[,4]

colnames(df_mosdepth_CCS_15kb_20kb_merged_1000_window_size_ONT_1000_combined) <- c("CHR", "START", "END", "CCS_DEPTH", "ONT_DEPTH")

df_mosdepth_CCS_15kb_20kb_merged_1000_window_size_ONT_1000_combined_elliptical_values <- df_mosdepth_CCS_15kb_20kb_merged_1000_window_size_ONT_1000_combined
medianccsdepth = median(mosdepth_CCS_15kb_20kb_merged_1000_window_size_chr_1_22[,"DEPTH"])
medianontdepth = median(mosdepth_ONT_1000_window_size_chr_1_22[,"DEPTH"])
df_mosdepth_CCS_15kb_20kb_merged_1000_window_size_ONT_1000_combined_elliptical_values[,6] <- sqrt(((df_mosdepth_CCS_15kb_20kb_merged_1000_window_size_ONT_1000_combined[,4]/medianccsdepth)^2 + (df_mosdepth_CCS_15kb_20kb_merged_1000_window_size_ONT_1000_combined[,5]/medianontdepth)^2)/2)
threshold_ellipctial_outlier = unname(quantile(df_mosdepth_CCS_15kb_20kb_merged_1000_window_size_ONT_1000_combined_elliptical_values[,6])[4]+(1.5*IQR(df_mosdepth_CCS_15kb_20kb_merged_1000_window_size_ONT_1000_combined_elliptical_values[,6])[1]))

df_mosdepth_CCS_15kb_20kb_merged_1000_window_size_ONT_1000_combined_outliers <- df_mosdepth_CCS_15kb_20kb_merged_1000_window_size_ONT_1000_combined_elliptical_values[which(df_mosdepth_CCS_15kb_20kb_merged_1000_window_size_ONT_1000_combined_elliptical_values[,6] > 1.360939),]

write.csv(df_mosdepth_CCS_15kb_20kb_merged_1000_window_size_ONT_1000_combined_outliers, file = "union_HG002_CCS_15kb_20kb_merged_ONT_1000_window_size_combined_elliptical_outlier_threshold.bed", row.names = FALSE)


### Alignment of assemblies to identify potential CNV

Nancy Hansen provided an both assembly coverage bedgraphs of maternal and paternal haplotypes in December 2019

In [ ]:
# HG2_SKor_TrioONTCanu
unionBedGraphs -i HG2_SKor_TrioONTCanuMat_1119_l100c500.qdelta.sort.renamedcontigs.bedgraph HG2_SKor_TrioONTCanuPat_1119_l100c500.qdelta.sort.renamedcontigs.bedgraph > HG2_SKor_TrioONTCanu_1119_l100c500.qdelta.sort.renamedcontigs.tempunion.bedgraph

python find_union.py --input HG2_SKor_TrioONTCanu_1119_l100c500.qdelta.sort.renamedcontigs.tempunion.bedgraph --output HG2_SKor_TrioONTCanu_1119_l100c500.union.qdelta.sort.renamedcontigs.bedgraph

python remove_all_but_chr1_22.py --input HG2_SKor_TrioONTCanu_1119_l100c500.union.qdelta.sort.renamedcontigs.bedgraph  --output HG2_SKor_TrioONTCanu_1119_l100c500.union.qdelta.sort.renamedcontigs.chr1_22.bedgraph

# HG2_SKor_TrioONTFlye
unionBedGraphs -i HG2_SKor_TrioONTFlyeMat_1119_l100c500.qdelta.sort.renamedcontigs.bedgraph HG2_SKor_TrioONTFlyePat_1119_l100c500.qdelta.sort.renamedcontigs.bedgraph > HG2_SKor_TrioONTFlye_1119_l100c500.qdelta.sort.renamedcontigs.tempunion.bedgraph

python find_union.py --input HG2_SKor_TrioONTFlye_1119_l100c500.qdelta.sort.renamedcontigs.tempunion.bedgraph --output HG2_SKor_TrioONTFlye_1119_l100c500.union.qdelta.sort.renamedcontigs.bedgraph

python remove_all_but_chr1_22.py --input HG2_SKor_TrioONTFlye_1119_l100c500.union.qdelta.sort.renamedcontigs.bedgraph --output HG2_SKor_TrioONTFlye_1119_l100c500.union.qdelta.sort.renamedcontigs.chr1_22.bedgraph

# HG2_SKor_CCS15
unionBedGraphs -i HG2_SKor_CCS15Mat_0918_l100c500.qdelta.sort.renamedcontigs.bedgraph HG2_SKor_CCS15Pat_0918_l100c500.qdelta.sort.renamedcontigs.bedgraph > HG2_SKor_CCS15_0918_l100c500.qdelta.sort.renamedcontigs.tempunion.bedgraph

python find_union.py --input HG2_SKor_CCS15_0918_l100c500.qdelta.sort.renamedcontigs.tempunion.bedgraph --output HG2_SKor_CCS15_0918_l100c500.union.qdelta.sort.renamedcontigs.bedgraph

python remove_all_but_chr1_22.py --input HG2_SKor_CCS15_0918_l100c500.union.qdelta.sort.renamedcontigs.bedgraph --output HG2_SKor_CCS15_0918_l100c500.union.qdelta.sort.renamedcontigs.chr1_22.bedgraph

## Intersect and find > 10kb

intersectBed -wa -a HG2_SKor_TrioONTCanu_1119_l100c500.union.qdelta.sort.renamedcontigs.chr1_22.bedgraph -b HG2_SKor_CCS15_0918_l100c500.union.qdelta.sort.renamedcontigs.chr1_22.bedgraph | sort -k1,1 -k2,2n - | bedtools intersect -a stdin -b HG2_SKor_TrioONTFlye_1119_l100c500.union.qdelta.sort.renamedcontigs.chr1_22.bedgraph | sort -k1,1 -k2,2n - | mergeBed -i stdin -d 100  | awk '$3-$2>10000' > HG2_SKor_TrioONTCanu_intersect_HG2_SKor_TrioONTFlye_intersect_HG2_SKor_CCS15_gt10kb_GRCh37.bed

find_union.py

import argparse

parser = argparse.ArgumentParser(description="Subset bed file to >1 for column 4")
parser.add_argument('--input', metavar="I", type=str, nargs="+", help="input bed file")
parser.add_argument('--output', metavar="O", type=str, nargs="+", help="output file")
args = parser.parse_args()

f = open(args.input[0], "r") 
f_lines = f.readlines()

f_out = open(args.output[0], "w+")
number_entries_covered = 0
for line in f_lines:
    split_line = line.split("\t")
    if int(split_line[3].strip()) > 1 or int(split_line[4].strip()) > 1:
        number_entries_covered = number_entries_covered + int(split_line[3])
        f_out.write(line)
        f_out.flush()

f.close()
f_out.close()


remove_all_but_chr1_22.py 

import argparse

parser = argparse.ArgumentParser(description="Remove all but chr1-22 entries")
parser.add_argument('--input', metavar="I", type=str, nargs="+", help="input file")
parser.add_argument('--output', metavar="O", type=str, nargs="+", help="output file")
args = parser.parse_args()

f = open(args.input[0], "r") 
f_lines = f.readlines()
chrs_to_keep = ["chr1","chr2","chr3","chr4","chr5","chr6","chr7","chr8","chr9","chr10","chr11","chr12","chr13","chr14","chr15","chr16","chr17","chr18","chr19","chr20","chr21","chr22"]

f_out = open(args.output[0], "w+")

for line in f_lines:
    split_line = line.split("\t")
    if split_line[0] in chrs_to_keep:
        f_out.write(line)
        f_out.flush()

f.close()
f_out.close()

### Inversions 

Nancy Hansen provided a list of inversions that were identified in HG002 in December 2019

In [ ]:
bedtools slop -i hsd37d5.112719.inversions.bed -g human.b37.genome -b 0.25 -pct > hsd37d5.112719.inversions_slop150.bed

python remove_all_but_chr1_22.py --input hsd37d5.112719.inversions_slop150.bed --output hsd37d5.112719.inversions_slop150_chr1_22.bed


convert_inversions_vcf_to_bed.py

import argparse

parser = argparse.ArgumentParser(description="Convert inversions VCF to bed")
parser.add_argument('--input', metavar="I", type=str, nargs="+", help="input bed file")
parser.add_argument('--output', metavar="O", type=str, nargs="+", help="output bed file")
args = parser.parse_args()

f = open(args.input[0], "r") 
f_out = open(args.output[0], "w+")

lines = f.readlines()

for line in lines:
    if "#" in line:
        continue
    split_line = line.split("\t")
    info_field = split_line[7]
    info_field_split = info_field.split(";")
    chrom = "0"
    start = "0"
    end = "0"
    clusterIDs = ""
    for split in info_field_split:
        if "REFWIDENED" in split:
            refwidened_split = split.split("=")
            refwidened_split_on_colon = refwidened_split[1].split(":")
            chrom = refwidened_split_on_colon[0]
            refwidened_split_start_end = refwidened_split_on_colon[1].split("-")
            start = refwidened_split_start_end[0]
            end = refwidened_split_start_end[1]
        elif "ClusterIDs" in split:
            clusterIDs = split
    to_write_out = chrom + "\t" + start + "\t" + end + "\t" + clusterIDs + "\n"
    f_out.write(to_write_out)
    f_out.flush()

f.close()
f_out.close()

### Use Aaron's VDJ regions file

Aaron Wenger's produced a bed file (hg19.vdj.bed.gz) that covers the VDJ and the README file he provided includes commands for generating it.

### Find L1Hs greater than 500 base pairs

Lines 1-3 are from Aaron Wenger's manual curation BEDs hg19 README.

In [ ]:
#download file from https://hgdownload.cse.ucsc.edu/goldenPath/hg19/database/rmsk.txt.gz

cat rmsk.txt | awk '{ print $6 "\t" $7 "\t" $8 "\t" $12 ":" $11 "\t" $10; }' > hg19.rmsk.bed

grep "L1H" hg19.rmsk.bed > hg19.rmsk.L1H.bed

python filter_size_gt.py --input hg19.rmsk.L1H.bed --output hg19.rmsk.L1H_gt500.bed --filter 500


filter_size_gt.py

import argparse

parser = argparse.ArgumentParser(description="Filter regions based on size")
parser.add_argument('--input', metavar="I", type=str, nargs="+", help="input file")
parser.add_argument('--output', metavar="O", type=str, nargs="+", help="output file")
parser.add_argument('--filter', metavar="F", type=str, nargs="+", help="filter file")
args = parser.parse_args()

f = open(args.input[0], "r") 
f_lines = f.readlines()

f_out = open(args.output[0], "w+")
size_filter = int(args.filter[0])
for line in f_lines:
    line_split = line.split("\t")
    start = int(line_split[1])
    end = int(line_split[2])
    size = end - start
    if size > size_filter:         
        f_out.write(line)
        f_out.flush()

    
f.close()
f_out.close()

### Generate segdups_chr1_22_gte_10kb_identity_gte_990_segdups_counts_gt_5.bed

Line 1 of these commands was adopted from Aaron Wenger's scripts to generate manual curation BED files.

In [ ]:
#download file from http://hgdownload.cse.ucsc.edu/goldenPath/hg19/database/genomicSuperDups.txt.gz

python remove_all_but_chr1_22_w_chr.py --input hg19.segdups.bed --output hg19.segdups_chr1_22.bed

python filter_segdup_size_bed.py --input hg19.segdups_chr1_22.bed --output1 hg19.segdups_chr1_22_gte_10kb.bed --output2 hg19.segdups_chr1_22_lt_10kb.bed --filter 10000  

python filter_percent_identity_bed.py --input hg19.segdups_chr1_22_gte_10kb.bed --output1 hg19.segdups_chr1_22_gte_10kb_identity_gte_990.bed --output2 hg19.segdups_chr1_22_gte_10kb_identity_lt_990.bed --filter 990    

sort -k 1,1 hg19.segdups_chr1_22_gte_10kb_identity_gte_990.bed > hg19.segdups_chr1_22_gte_10kb_identity_gte_990.sorted_for_genomecov.bed

bedtools genomecov -bg -i hg19.segdups_chr1_22_gte_10kb_identity_gte_990.sorted_for_genomecov.bed -g GRCh37.genome > hg19.segdups_chr1_22_gte_10kb_identity_gte_990_segdups_counts.bed

python filter_count_gt_bed.py --input hg19.segdups_chr1_22_gte_10kb_identity_gte_990_segdups_counts.bed --output hg19.segdups_chr1_22_gte_10kb_identity_gte_990_segdups_counts_gt_5.bed --filter 5


filter_segdup_size_bed.py

import argparse

parser = argparse.ArgumentParser(description="Filter segdups based on size")
parser.add_argument('--input', metavar="I", type=str, nargs="+", help="input file")
parser.add_argument('--output1', metavar="O", type=str, nargs="+", help="output file 1")
parser.add_argument('--output2', metavar="U", type=str, nargs="+", help="output file 2")
parser.add_argument('--filter', metavar="F", type=str, nargs="+", help="filter file")
args = parser.parse_args()

f = open(args.input[0], "r") 
f_lines = f.readlines()

f_out1 = open(args.output1[0], "w+")
f_out2 = open(args.output2[0], "w+")
size_filter = int(args.filter[0])
for line in f_lines:
    line_split = line.split("\t")
    segdup_start = int(line_split[1])
    segdup_end = int(line_split[2])
    segdup_size = segdup_end - segdup_start
    if segdup_size >= size_filter:         
        f_out1.write(line)
        f_out1.flush()
    elif segdup_size < size_filter:
        f_out2.write(line)
        f_out2.flush()
    
f.close()
f_out1.close()
f_out2.close()

filter_percent_identity_bed.py

import argparse

parser = argparse.ArgumentParser(description="Filter segdups based on percent identity")
parser.add_argument('--input', metavar="I", type=str, nargs="+", help="input file")
parser.add_argument('--output1', metavar="O", type=str, nargs="+", help="output file 1")
parser.add_argument('--output2', metavar="U", type=str, nargs="+", help="output file 2")
parser.add_argument('--filter', metavar="F", type=str, nargs="+", help="filter file")
args = parser.parse_args()

f = open(args.input[0], "r") 
f_lines = f.readlines()

f_out1 = open(args.output1[0], "w+")
f_out2 = open(args.output2[0], "w+")
size_filter = int(args.filter[0])
for line in f_lines:
    line_split = line.split("\t")
    segdup_percent_identity = int(line_split[5])
    if segdup_percent_identity >= size_filter:         
        f_out1.write(line)
        f_out1.flush()
    elif segdup_percent_identity < size_filter:
        f_out2.write(line)
        f_out2.flush()
    
f.close()
f_out1.close()
f_out2.close()

filter_count_gt_bed.py

import argparse

parser = argparse.ArgumentParser(description="Filter regions by count")
parser.add_argument('--input', metavar="I", type=str, nargs="+", help="input file")
parser.add_argument('--output', metavar="O", type=str, nargs="+", help="output file")
parser.add_argument('--filter', metavar="F", type=str, nargs="+", help="filter file")
args = parser.parse_args()

f = open(args.input[0], "r") 
f_lines = f.readlines()

f_out = open(args.output[0], "w+")
size_filter = int(args.filter[0])
for line in f_lines:
    line_split = line.split("\t")
    segdup_count = int(line_split[3])
    if segdup_count > size_filter:         
        f_out.write(line)
        f_out.flush()

    
f.close()
f_out1.close()

### Expand gaps by 15,000 base pairs

In [ ]:
bedtools slop -i example_of_no_ref_regions_input_file_b37.bed -g genome.dict -b 15000 | mergeBed -i stdin > example_of_no_ref_regions_input_file_b37_bedtools_slop_15kb_merged.bed



### Expand GIAB SV v0.6 by 50%

In [ ]:
bedtools slop -i HG002_SVs_Tier1plusTier2_v0.6.1.bed -g genome.dict -b .25 -pct | mergeBed -i stdin > expanded_150_HG002_SVs_Tier1plusTier2_v0.6.1.bed



### Regions covered by no contig in 38 to 37 alignment and regions covered by more than 1 contigs

In [ ]:
#generate coverage of genome regions by 38 to 37 alignment
/Volumes/DroboZook/bioinfo/bedtools2.25.0/bin/genomeCoverageBed -ibam /Volumes/giab/data/reference_genomes/refrefalignments/hs37d5_hg38_minimap2_asm20_N10.bam -bg -g /Volumes/DroboZook/bioinfo/bedtools2.25.0/genomes/human.b37.genome.txt > /Volumes/giab/data/reference_genomes/refrefalignments/hs37d5_hg38_minimap2_asm20_N10_coverage.bed
#regions larger than 1kb, after merging gaps with 100bp, covered by >1 GRCh38 contig
awk '$4>1' /Volumes/giab/data/reference_genomes/refrefalignments/hs37d5_hg38_minimap2_asm20_N10_coverage.bed | /Volumes/DroboZook/bioinfo/bedtools2.25.0/bin/mergeBed -i stdin -d 100 | awk '$3-$2>1000' > /Volumes/giab/data/reference_genomes/refrefalignments/hs37d5_hg38_minimap2_asm20_N10_gt1contig_gt1kb.bed

### Find assembly contigs less than 500kb

Lines 1-7 are from Aaron Wenger's analysis of singly unique nucleotides in v4.

In [ ]:
# download gap track from ftp://hgdownload.cse.ucsc.edu/goldenPath/hg19/database/gap.txt.gz ; convert to BED
cat gap.txt | cut -f2-4,8 | awk '{n=$4; if(n=="clone" || n=="contig" || n=="scaffold" || n=="short_arm") { n="gap"; } print $1"\t"$2"\t"$3"\t"n; }' | sort -k1,1 -k2,2g > hg19.gap.bed

# complement to identify contigs; ignore random chromosomes. ignoring 'alt' and 'fix' are new for GRCh37. Create BED with score set to contig length.
grep -v -e 'hap' hg19.gap.bed > hg19.gap_no_random.bed
cat genome.fa.fai | sort -k1,1 > hg19_sorted.fa.fai 
bedtools complement -g hg19_sorted.fa.fai -i hg19.gap_no_random.bed | awk '($1>=1 && $1<=22) { print $0 "\tcontig\t" $3-$2; }' > hg19.contigs.1-22.bed

awk '($5 < 5e5)' hg19.contigs.1-22.bed > hg19.contigs.1-22.lt_500kb.bed


### SP_SPonly regions

In [ ]:
Valerie Schnieder pointed out there are GRCh37 alignments to GRCh38 with regions resolved and the type. 

Information can be found at NCBI Assm-Assm alignments explained https://www.ncbi.nlm.nih.gov/genome/tools/remap/docs/alignments

The GRCh37 aligned to GRCh38 (excluding alts): ftp://ftp.ncbi.nlm.nih.gov/pub/remap/Homo_sapiens/2.1/GCF_000001405.13_GRCh37/GCF_000001405.26_GRCh38/

For the differences between the files these types were of interest: Discrepancy = SP : regions that are collapsed Discrepancy = SP-only : regions that are expanded

At that ftp link, the file GCF_000001405.13.xlxs was parsed to create a bed file containing SP and SP-only regions named SP_SPonly_primary_assm_alignments_sorted_merged.bed